# Week 05: Advanced NumPy

In [1]:
import numpy as np

## Advanced Indexing

Last week's lecture covered what's known as basic indexing, which, aside from the multidimensional aspect is mostly equivalent to Python's slicing. Advanced indexing (or sometimes called fancy indexing) is something unique to NumPy and behaves differently. It is triggered when the selection object (i.e. whatever is written inside the square index brackets) is 

 - a non-tuple sequence object (e.g. a list)
 - a `np.ndarray`
 - or a tuple with at least one element being one of the above
 
Note that in both Python and NumPy, `x[(exp1, exp2, ..., expN)]` is the same as `x[exp1, exp2, ..., expN]`, and in both cases the selection object is a tuple. 
 
Unlike basic indexing, advanced indexing always returns a **copy** instead of a view.

In [3]:
array = np.arange(9).reshape(3, 3)

In [8]:
# these lines will trigger basic indexing

array[1, 2]
array[(1, 2)]

# these lines will trigger advanced indexing

array[[1, 2]]

index = np.arange(3)
array[index]

array[[1, 2], 0];

### Integer Array Indexing

In the most straightforward case of advanced indexing, we supply a 1D integer array (any int-type works) per dimension of the array we want to index (`array`). To keep it simple, let's for now assume all arrays have the same shape `(n, )`.

This will then select `n` elements from `array`, where the $i$-th element's indices are the $i$-th elements of all the index arrays.

In [11]:
array = np.arange(9).reshape(3, 3)

index_row = np.array([0, 1, 2])
index_col = np.array([2, 2, 0])

# this will get elements at (0, 2), (1, 2) and (2, 0)
print(array[index_row, index_col])

index = np.zeros(10, dtype=int)

print(array[index, index])

[2 5 6]
[0 0 0 0 0 0 0 0 0 0]


**The actual behavior is a bit more general:**

The index arrays don't actually have to be 1D but instead can have *any* shape `s`. This will result in the returned array also having shape `s`.

Furthermore, the index arrays don't have to be the same shape, it is sufficient that *broadcasting* can be performed between them (remember last week's section on broadcasting rules). The result of the broadcasting step is what determines the shape of the returned array.

Some examples to make this clear:

In [13]:
array = np.arange(9).reshape(3, 3)

index_row = np.array([[0, 1], [0, 1]])
index_col = np.array([[0, 0], [1, 1]])

# this will select elements at (0, 0), (1, 0), (0, 1) and (1, 1)
# the output shape will be (2, 2), because that's the shape of both index arrays
print(array[index_row, index_col], "\n")

# this will select row 0, then 1, then 0 and then 1 again
# for columns, it will always select 0 because it gets broadcast to a (2, 2) shape
# the output shape will still be (2, 2), because that's the result of the broadcasting
print(array[index_row, 0])

[[0 3]
 [1 4]] 

[[0 3]
 [0 3]]


It should be noted that Python lists are treated the same way as NumPy arrays when it comes to advanced indexing, except for the case below:

### Special Case: Single Nested List

What happens when the selection object is a single nested list?

In [16]:
array = np.arange(9).reshape(3, 3)

array[[[0, 1], [0, 1]]]

<ipython-input-16-b030826cbb6b>:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  array[[[0, 1], [0, 1]]]


array([0, 4])

The answer is that the rules what happens will change in some future version of NumPy. Currently, it is interpreted as if the list was a tuple containing multiple advanced indices. As the warning above will tell you however, this will be changed to a more consistent interpretation, where the list is treated as a single, multidimensional advanced index for only the first dimension.

### Combining Advanced Indexing and Slicing

You can combine slices and advanced indexing in the same operation. There are two major cases, upon which the behavior depends drastically:
 - The advanced indices are all next to each other in the square brackets, for example: `x[..., arr1, arr2, :]`. This will result in what you would probably expect intuitively, the dimensions from the index arrays are inserted into the position they are in in the selection object, replacing the original array's dimensions.
 
 - The advanced indices are **not** all next to each other but separated by a slice, `...` or `np.newaxis`, for example: `x[arr1, :, arr2]`. In this case, there is no unambiguous spot to insert the dimensions from the index arrays, and thus they are tacked-on to the beginning.
 
Usually, you will get by without having to combine basic and advanced indexing. However, if you need it and have trouble understanding the results you get, you can check the following section from the NumPy documentation for a slightly more thorough explanation and some examples: https://numpy.org/doc/stable/reference/arrays.indexing.html#combining-advanced-and-basic-indexing

### Examples

#### Using `np.argsort` for advanced indexing

`np.argsort`, like `np.sort`, will sort an array. However, instead of the sorted elements, it will return the sorted **indices** as an array. This can be useful in conjunction with advanced indexing: Let's sort the values of one array according to the values of a different array.

In [17]:
letters = np.array(["y", "P", "n", "o", "t", "h"])
order = np.array([2, 1, 6, 5, 3, 4])

letters[np.argsort(order)]

array(['P', 'y', 't', 'h', 'o', 'n'], dtype='<U1')

####  some other great example perhaps?

tournament, all play all, but only once --> indexing with triangular matrix

### `np.nonzero()`

`array.nonzero()` (or `np.nonzero(array)`) returns the indices of all elements in an array that are $\neq$ 0 (that includes `True`, but not `False`). The output format is a tuple of $n$ 1D arrays, where $n$ is the number of dimensions of `array`. This "happens" to be the appropriate format for advanced indexing. 

In [19]:
array = np.array([[0, 2], [0, -1]])

array.nonzero()

(array([0, 1]), array([1, 1]))

Often this is used with boolean arrays (note that `array < 0` *is* a boolean array):

In [20]:
(array < 0).nonzero()

(array([1]), array([1]))

As mentioned, the result of `nonzero()` can be used for advanced indexing, although this simple example below is not actually a good use for that. It will in fact return all elements that are non-zero, but the better way of handling this is using boolean indexing, as will be discussed next.

In [21]:
array[array.nonzero()]

array([ 2, -1])

### Boolean Array Indexing

The *other* type of advanced indexing occurs when the index arrays (or lists) have type `bool` instead of an integer type. In this case, the shape of the index arrays is more restricted - it needs to match the dimension(s) it is trying to index.

One way to think about boolean indices is as an *overlay* over the original array. Only those values where the overlay has the value `True` are returned. The return shape of boolean index operations is always 1D, except when mixed with other types of indexing. In that case, more complicated rules similar to the ones described above apply. 

In [24]:
array = np.array([[0, 2], [0, -1]])

index = np.array([[True, False], [False, True]])
index.dtype

dtype('bool')

In [25]:
array[index]

array([ 0, -1])

Coming back to the `array.nonzero()` example, here finally is the usual way to find all non-zero elements. Boolean indexing tends to perform better.

In [27]:
array[array != 0]

array([ 2, -1])

In [28]:
n = 10_000
large_array = np.arange(n * n).reshape(n, n)

%timeit large_array[large_array.nonzero()]
%timeit large_array[large_array != 0]

3.89 s ± 848 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
682 ms ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Bitwise Operators

If you want to perform element-wise logical operations (like and, or, xor) on NumPy arrays, you can use the bitwise operators:

| logical operation | operator |
|:------------------|:--------:|
| and               | `&`      |
| or                | `\|`     |
| exclusive or      | `^`      |
| not               | `~`      |

Make sure to mind the order of operations, as bitwise operators in Python bind more strongly than comparison operators. Usually you want parentheses, as in the example below.

In [29]:
array = np.array([[0, 2], [0, -1]])

array[(array < 0) | (array > 0)]

array([ 2, -1])

## Value Assignment

Both types of advanced indexing can of course not only be used to get values from the array, but also to assign values. Besides the correct datatype, the value to be assigned must have a shape that is broadcastable to the shape defined by the index.

In [54]:
array = np.arange(25).reshape(5, 5)

array[array % 2 == 0] = 0

print(array)

[[ 0  1  0  3  0]
 [ 5  0  7  0  9]
 [ 0 11  0 13  0]
 [15  0 17  0 19]
 [ 0 21  0 23  0]]


## Concatenating Arrays

NumPy provides confusingly many functions for concatenating arrays, but they all build on `np.concatenate`.

It takes a tuple of arrays (of appropriate shape) and concatenates them along a given (already existing) axis.

Keep in mind that due to the fixed nature of arrays, concatenation is **slow**. If you know in advance what size an array needs to be in the end, it is pretty much always faster to assign it that size at creation.

In [33]:
array1 = np.arange(4).reshape(2, 2)
array2 = array1 + 4

print(np.concatenate((array1, array2), axis=0), "\n")
print(np.concatenate((array1, array2), axis=1))

[[0 1]
 [2 3]
 [4 5]
 [6 7]] 

[[0 1 4 5]
 [2 3 6 7]]


A fancy shorthand is the `np.r_` object. It does not provide any functionality that cannot be achieved using `concatenate`, but is often shorter. It also serves as an introduction to a couple of these objects in the NumPy library that are interacted with using indexing syntax instead of function call syntax.

 - the first element of the "selection tuple" is an optional string, specifying the axis of concatenation
 - the remaining elements are either 
     - arrays to be concatenated, or
     - slices that roughly represent a call to `np.arange` or `np.linspace`

In [46]:
print(np.r_[array1, array2], "\n")       # simple tuple of arrays to concatenate (axis 0 is default)
print(np.r_["1", array1, array2], "\n")  # now with the optional axis string

print(np.r_[1:10, 10:0:-2])              # slices normally represent np.arange calls (start:stop:step_size) ...
print(np.r_[1:10, 10:0:20j])             # ... unless the third part of the slice is a complex number
                                         # in that case, it represents the *number* of steps inbetween

[[0 1]
 [2 3]
 [4 5]
 [6 7]] 

[[0 1 4 5]
 [2 3 6 7]] 

[ 1  2  3  4  5  6  7  8  9 10  8  6  4  2]
[ 1.          2.          3.          4.          5.          6.
  7.          8.          9.         10.          9.47368421  8.94736842
  8.42105263  7.89473684  7.36842105  6.84210526  6.31578947  5.78947368
  5.26315789  4.73684211  4.21052632  3.68421053  3.15789474  2.63157895
  2.10526316  1.57894737  1.05263158  0.52631579  0.        ]


There is one conceptually slightly different case: Concatenation along a **new** axis. In that case, it's easiest to use `np.stack`. The `axis` parameter here specifies where to insert the new axis.

In [52]:
np.stack((array1, array2), axis=0)

array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])

## Coordinate Grids

Coordinate grids are a useful tool to have when evaluating a multidimensional function over a range of values. This is especially useful for plotting purposes, and we will come back to that in one of the plotting lectures. For now, let's just look at how to use `np.meshgrid` to create a coordinate grid.

Suppose we want to evaluate `function(x, y)` for `x = [1, 2, 3]` and `y = [7, 8, 9]`. What this really means is that we need all possible combinations of `x` and `y`, making a total of 9 pairs. The output should be a 3x3 array containing all the results. That means the two input arrays, let's call them `xx` and `yy` also need to be of 3x3 shape. `np.meshgrid` was made for exactly that purpose!

In [53]:
x = np.array([1, 2, 3])
y = np.array([7, 8, 9])

xx, yy = np.meshgrid(x, y)

print(xx, "\n")
print(yy, "\n")

[[1 2 3]
 [1 2 3]
 [1 2 3]] 

[[7 7 7]
 [8 8 8]
 [9 9 9]] 



A similar, but simpler function is `np.indices(shape)`. For a given shape of array, it returns all possible indices. This is again in a format directly appropriate for advanced indexing. 

In [66]:
ix_row, ix_col = np.indices((3, 3))

array = np.arange(9).reshape(3, 3)

print(ix_row, "\n")
print(ix_col, "\n")

print(array[ix_row, ix_col])

[[0 0 0]
 [1 1 1]
 [2 2 2]] 

[[0 1 2]
 [0 1 2]
 [0 1 2]] 

[[0 1 2]
 [3 4 5]
 [6 7 8]]


### Creating `ufunc`s

With `np.vectorize` NumPy provides a convenient way to convert any Python function that works with single values to a NumPy `ufunc`. Recall, this means that the function will then accept entire NumPy arrays as input and even perform broadcasting when required.

Note that this is purely for convenience, it does not magically improve performance of those functions.

In [67]:
import math

def my_function(a, b):
    # this function would normally not work with NumPy arrays
    # because we are using the built-in math module
    return math.sin(a) + math.cos(b)
    
my_function_numpy = np.vectorize(my_function)

a = np.linspace(0, 2, 10)
b = np.linspace(0, -4, 10)

my_function_numpy(a, b)

array([1.00000000e+00, 1.12324741e+00, 1.06023141e+00, 8.53607376e-01,
       5.70865201e-01, 2.89871279e-01, 8.26113332e-02, 3.48387941e-04,
       6.31217287e-02, 2.55653806e-01])

## What's Next?

While researching for this lecture, I've been repeatedly surprised by how deep the NumPy rabbit hole goes, even though I thought I had a pretty decent understanding already. Many features and technicalities of NumPy did not make it into this lecture, but you should have a good foundation now to work somewhat efficiently in NumPy, as well as look up features on your own. "How to do \<thing> in NumPy" is always a good idea to type into Google!

In [58]:
np.indices(array.shape)

array([[[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]]])

In [72]:
np.set_printoptions(formatter={"bool":lambda b: "x" if b else "o"})

In [73]:
print(array % 2 == 0)

[[x o x]
 [o x o]
 [x o x]]
